In [1]:
import sys
import os
import shutil
import datetime
import re
import pandas as pd
from collections import Counter
#sys.path.append(os.path.abspath("/home/jose/Dropbox/Doktorarbeit/"))
#from reading_robot import load_data, tei2text, text2features, classify, process_labels, describe_data,feature_analysis, cull_data, st_tests, metadata2numbers, cluster, call_stylo, prepare_subcorpus
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
%matplotlib inline
from shutil import copyfile
import numpy as np
import glob

# Read Data

In [2]:
wdir = "../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base/"

In [3]:
files = glob.glob(wdir + "/*.txt")


In [4]:
files

['../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\AgustinSalazar_Sonetos1.txt',
 '../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\AgustinSalazar_Sonetos2.txt',
 '../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\AntonioEnriquezGomez_Sonetos1.txt',
 '../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\AntonioEnriquezGomez_Sonetos2.txt',
 '../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\BartolomeArgensola_Sonetos1.txt',
 '../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\BartolomeArgensola_Sonetos2.txt',
 '../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\BernardinodeRebolledo_Sonetos1.txt',
 '../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\BernardinodeRebolledo_Sonetos2.txt',
 '../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\BocangelyUnzueta_Sonetos1.txt',
 '../data/corpus_Poesia_SiglodeOro/only-tokens/directorio_base\\BocangelyUnzueta_Sonetos2.txt',
 '../data/corpus_Poesi

In [5]:
files_df = pd.DataFrame(files, columns=["path"])

In [6]:
files_df

,path
0,../data/corpus_Poesia_SiglodeOro/only-tokens/d...
1,../data/corpus_Poesia_SiglodeOro/only-tokens/d...
2,../data/corpus_Poesia_SiglodeOro/only-tokens/d...
3,../data/corpus_Poesia_SiglodeOro/only-tokens/d...
4,../data/corpus_Poesia_SiglodeOro/only-tokens/d...
...,...
77,../data/corpus_Poesia_SiglodeOro/only-tokens/d...
78,../data/corpus_Poesia_SiglodeOro/only-tokens/d...
79,../data/corpus_Poesia_SiglodeOro/only-tokens/d...
80,../data/corpus_Poesia_SiglodeOro/only-tokens/d...


In [7]:
files_df["file_name"] = files_df["path"].str.findall("\\\(.*?)\.txt").str[0]

In [8]:
files_df

,path,file_name
0,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AgustinSalazar_Sonetos1
1,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AgustinSalazar_Sonetos2
2,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AntonioEnriquezGomez_Sonetos1
3,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AntonioEnriquezGomez_Sonetos2
4,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,BartolomeArgensola_Sonetos1
...,...,...
77,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,SorJuanaInesdelaCruz_Sonetos2
78,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TirsodeMolina_Sonetos1
79,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TirsodeMolina_Sonetos2
80,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TrilloyFigueroa_Sonetos1


In [9]:
files_df["author"] = files_df["file_name"].str.findall("^(.*?)_").str[0]

In [10]:
files_df["text_title"] = files_df["file_name"].str.findall("_(.*?)$").str[0]

In [11]:
files_df

,path,file_name,author,text_title
0,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AgustinSalazar_Sonetos1,AgustinSalazar,Sonetos1
1,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AgustinSalazar_Sonetos2,AgustinSalazar,Sonetos2
2,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AntonioEnriquezGomez_Sonetos1,AntonioEnriquezGomez,Sonetos1
3,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AntonioEnriquezGomez_Sonetos2,AntonioEnriquezGomez,Sonetos2
4,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,BartolomeArgensola_Sonetos1,BartolomeArgensola,Sonetos1
...,...,...,...,...
77,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,SorJuanaInesdelaCruz_Sonetos2,SorJuanaInesdelaCruz,Sonetos2
78,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TirsodeMolina_Sonetos1,TirsodeMolina,Sonetos1
79,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TirsodeMolina_Sonetos2,TirsodeMolina,Sonetos2
80,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TrilloyFigueroa_Sonetos1,TrilloyFigueroa,Sonetos1


In [12]:
sorted(list(files_df.author.unique()))


['AgustinSalazar',
 'AntonioEnriquezGomez',
 'BartolomeArgensola',
 'BernardinodeRebolledo',
 'BocangelyUnzueta',
 'Cervantes',
 'DiegoHurtadodeMendoza',
 'DiegoXimenezAyllon',
 'FranciscoDelaTorre',
 'FranciscodeAldana',
 'FranciscodeBorja',
 'FranciscodeFigueroa',
 'FranciscodeMedrano',
 'Garcilaso',
 'Gongora',
 'GutierredeCetina',
 'HernandodeAcunya',
 'Herrera',
 'JosephdeLitalayCastelvi',
 'JuanBoscan',
 'JuandeAlmeida',
 'JuandeArguijo',
 'JuandeJauregui',
 'JuandeTassisyPeralta',
 'JuandeTimoneda',
 'LopedeVega',
 'LopedeZarate',
 'LuisCarrilloySotomayor',
 'LuisMartindelaPlaza',
 'LuisdeUlloayPereira',
 'LupercioArgensola',
 'MiradeAmescua',
 'Pacheco',
 'PedroSotodeRojas',
 'PedrodeEspinosa',
 'PedrodePadilla',
 'PolodeMedina',
 'Quevedo',
 'SorJuanaInesdelaCruz',
 'TirsodeMolina',
 'TrilloyFigueroa']

In [13]:
for index, row in files_df.iterrows():
    with open(row["path"], "r", errors="replace", encoding="utf-8") as fin:
        content_str = fin.read()
    files_df.loc[index, "text"] = content_str

In [14]:
files_df

,path,file_name,author,text_title,text
0,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AgustinSalazar_Sonetos1,AgustinSalazar,Sonetos1,\nRosa\nde\nel\nprado\nestrella\nnacarada\nast...
1,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AgustinSalazar_Sonetos2,AgustinSalazar,Sonetos2,\nSi\nde\nalguna\ntaberna\nen\nlos\ntapices\nv...
2,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AntonioEnriquezGomez_Sonetos1,AntonioEnriquezGomez,Sonetos1,\nEn\nTirias\ntersa\nde\npurpúrea\npompa\nAmar...
3,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AntonioEnriquezGomez_Sonetos2,AntonioEnriquezGomez,Sonetos2,\nAlta\ncontemplación\ncómo\nes\nposible\nque\...
4,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,BartolomeArgensola_Sonetos1,BartolomeArgensola,Sonetos1,\nAunque\nen\ntus\nnaves\noh\nBretaña\ningrata...
...,...,...,...,...,...
77,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,SorJuanaInesdelaCruz_Sonetos2,SorJuanaInesdelaCruz,Sonetos2,\nFeliciano\nme\nadora\ny\nle\naborrezco\nLisa...
78,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TirsodeMolina_Sonetos1,TirsodeMolina,Sonetos1,\nCuán\nenvidiosa\ndulce\nprenda\nmía\nel\nalm...
79,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TirsodeMolina_Sonetos2,TirsodeMolina,Sonetos2,\nQué\nconfusión\nenmarañados\ncielos\nes\nest...
80,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TrilloyFigueroa_Sonetos1,TrilloyFigueroa,Sonetos1,\nA\nel\npie\nde\nuna\nalta\nhaya\nen\ndulce\n...


In [15]:
files_df.index = files_df.file_name

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
reg_ex = r"(?u)\b[\w_@]+\b|[¶\(»\]\?\.\–\!’•\|“\>\)\-\—\:\}\*\&…¿\/=¡\"\'·+\{\#\[;­,«~]"
sort_columns_by_frequency = True
lowercase = True
max_features = 1000000
analyzer = "word" #{‘word’, ‘char’, ‘char_wb’}
ngram_range = (1,1) #ngram_range = None

#vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(corpus)

#vectorizer = CountVectorizer()

vec = CountVectorizer(
        token_pattern = reg_ex,
        lowercase = lowercase,
        max_features = max_features,
        analyzer = analyzer,
        ngram_range = ngram_range,
        )

tokens = vec.fit_transform(files_df.text.tolist())

tokens_df = pd.DataFrame(tokens.toarray(), columns = vec.get_feature_names(), index = files_df["file_name"])

In [18]:
tokens_df

,',-,a,a_arquímedes,a_buena_cuenta,a_bulto,a_cada_instante,a_cada_paso,a_caria,a_causa_de,...,última,últimas,último,últimos,única,únicamente,único,útil,útilmente,ยกdichoso
file_name,,,,,,,,,,,,,,,,,,,,,
AgustinSalazar_Sonetos1,0,0,54,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AgustinSalazar_Sonetos2,0,0,34,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AntonioEnriquezGomez_Sonetos1,0,0,30,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
AntonioEnriquezGomez_Sonetos2,0,0,29,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BartolomeArgensola_Sonetos1,0,0,237,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SorJuanaInesdelaCruz_Sonetos2,0,0,87,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,0
TirsodeMolina_Sonetos1,0,0,82,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TirsodeMolina_Sonetos2,0,0,74,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
files_df["tokens_sum"] = tokens_df.sum(axis=1)

In [20]:
files_df

,path,file_name,author,text_title,text,tokens_sum
file_name,,,,,,
AgustinSalazar_Sonetos1,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AgustinSalazar_Sonetos1,AgustinSalazar,Sonetos1,\nRosa\nde\nel\nprado\nestrella\nnacarada\nast...,1444
AgustinSalazar_Sonetos2,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AgustinSalazar_Sonetos2,AgustinSalazar,Sonetos2,\nSi\nde\nalguna\ntaberna\nen\nlos\ntapices\nv...,1348
AntonioEnriquezGomez_Sonetos1,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AntonioEnriquezGomez_Sonetos1,AntonioEnriquezGomez,Sonetos1,\nEn\nTirias\ntersa\nde\npurpúrea\npompa\nAmar...,1800
AntonioEnriquezGomez_Sonetos2,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,AntonioEnriquezGomez_Sonetos2,AntonioEnriquezGomez,Sonetos2,\nAlta\ncontemplación\ncómo\nes\nposible\nque\...,1806
BartolomeArgensola_Sonetos1,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,BartolomeArgensola_Sonetos1,BartolomeArgensola,Sonetos1,\nAunque\nen\ntus\nnaves\noh\nBretaña\ningrata...,7438
...,...,...,...,...,...,...
SorJuanaInesdelaCruz_Sonetos2,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,SorJuanaInesdelaCruz_Sonetos2,SorJuanaInesdelaCruz,Sonetos2,\nFeliciano\nme\nadora\ny\nle\naborrezco\nLisa...,3380
TirsodeMolina_Sonetos1,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TirsodeMolina_Sonetos1,TirsodeMolina,Sonetos1,\nCuán\nenvidiosa\ndulce\nprenda\nmía\nel\nalm...,2641
TirsodeMolina_Sonetos2,../data/corpus_Poesia_SiglodeOro/only-tokens/d...,TirsodeMolina_Sonetos2,TirsodeMolina,Sonetos2,\nQué\nconfusión\nenmarañados\ncielos\nes\nest...,2653


In [21]:
files_df.author.value_counts().mean()

2.0

In [22]:
files_df.shape[0]

82

In [23]:
files_df.author.value_counts()

LuisCarrilloySotomayor     2
JosephdeLitalayCastelvi    2
BartolomeArgensola         2
JuandeArguijo              2
FranciscoDelaTorre         2
FranciscodeMedrano         2
FranciscodeFigueroa        2
DiegoHurtadodeMendoza      2
HernandodeAcunya           2
JuandeAlmeida              2
LuisMartindelaPlaza        2
PedrodePadilla             2
PedroSotodeRojas           2
FranciscodeBorja           2
FranciscodeAldana          2
JuanBoscan                 2
BernardinodeRebolledo      2
PolodeMedina               2
Quevedo                    2
TirsodeMolina              2
DiegoXimenezAyllon         2
JuandeTimoneda             2
Herrera                    2
BocangelyUnzueta           2
LopedeZarate               2
Gongora                    2
LupercioArgensola          2
TrilloyFigueroa            2
SorJuanaInesdelaCruz       2
JuandeJauregui             2
Garcilaso                  2
LopedeVega                 2
Pacheco                    2
AntonioEnriquezGomez       2
MiradeAmescua 

In [24]:
files_df.to_csv("./../data/corpus_Poesia_SiglodeOro/metadata.tsv", sep="\t")

In [25]:
number_texts = files_df.shape[0]
number_authors = len(files_df.author.unique())
mean_number_text_author = files_df.author.value_counts().mean()
std_number_text_author = files_df.author.value_counts().std()
mean_length_text = files_df.tokens_sum.mean()
std_length_text = files_df.tokens_sum.std()
distribution_authors_texts = files_df.author.value_counts()


In [26]:
summary_corpus_df = pd.DataFrame(
            [number_texts,   number_authors,  mean_number_text_author,   std_number_text_author,  mean_length_text, std_length_text, distribution_authors_texts  ],
    index=["number_texts","number_authors","mean_number_text_author", "std_number_text_author","mean_length","std_length", "distribution_authors_texts"  ],
    columns = ["16-17-poesia"]).T

In [27]:
summary_corpus_df

,number_texts,number_authors,mean_number_text_author,std_number_text_author,mean_length,std_length,distribution_authors_texts
16-17-poesia,82,41,2,0,4818.85,6630.09,LuisCarrilloySotomayor 2 JosephdeLitalayCa...


In [28]:
summary_corpus_df.to_csv("./../data/corpus_Poesia_SiglodeOro/summary.tsv", sep="\t")